In [1]:
import fnmatch
import pandas as pd

In [2]:
gatewayVolumesLoc = "./GatewayVolumes_2056.csv"
chainFlowVolumesLoc = "./INITFMCC_ChainFlowShare_Y2056_WFY_Funded_Op2_001.CSV"
postcodeSharesLoc = "./PostcodeShares_2056.csv"
travelTimesLoc = "./INITFMCC_PCTTPert_Y2056_WFY_Funded_Op2_001.CSV"

In [3]:
gatewayVolumes = pd.read_csv(gatewayVolumesLoc)
chainFlowVolumes = pd.read_csv(chainFlowVolumesLoc)
postcodeShares = pd.read_csv(postcodeSharesLoc)
postcodeShares['Postcode'] = postcodeShares['Postcode'].astype(str)
travelTimes = pd.read_csv(travelTimesLoc, low_memory=False)

gatewayMatchColNames = ['Gateway','Sector','Import / Export','Full / Empty']

In [14]:
chainFlowVolumes

,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Volume Entry,From,To,From State,To State,Run Type,Output Link Share
0,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442
1,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671
2,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977
3,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909
4,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.085009
...,...,...,...,...,...,...,...,...,...,...,...,...
233,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,Transport Depots,BIFT South Rail Terminal,Empty,Complete,IG,0.200000
234,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,BIFT South Rail Terminal,Exporters,Empty,Full,CR,0.082162
235,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,Exporters,BIFT South Rail Terminal,Full,Complete,CR,0.664516
236,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,Exporters,Transport Depots,Full,Full,CR,0.135484


In [4]:
travelTimes['From Postcode'] = travelTimes['From Postcode'].astype(str)
travelTimes['To Postcode'] = travelTimes['To Postcode'].astype(str)

In [5]:
def matchStrings(primaryString, secondaryString):
    isMatch = False
    secondaryStringList = secondaryString.split(';')
    for i in range(len(secondaryStringList)):
        isMatch = isMatch | fnmatch.fnmatch(primaryString, secondaryStringList[i])
    return isMatch

def matchCols(primaryRow, secondaryRow, primaryColsList, secondaryColsList):
    rowMatch = True
    for j in range(len(primaryColsList)):
        rowMatch = rowMatch & matchStrings(primaryRow[primaryColsList[j]], secondaryRow[secondaryColsList[j]])
    return rowMatch

In [13]:
from IPython.display import display  # For nicer DataFrame output

def pretty_print_df(df, title="DataFrame", max_rows=10):
    """Display a DataFrame with a title and limited rows"""
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    display(df.head(max_rows))
    print(f"Shape: {df.shape}")
    if len(df) > max_rows:
        print(f"Showing first {max_rows} rows of {len(df)} total rows")

allTravelTimes = pd.DataFrame()

for gatewayIt in range(len(gatewayVolumes)):
    gatewayRow = gatewayVolumes.iloc[gatewayIt]
    print('\nCurrent gateway:' + str(gatewayRow.values) + '\n')

    matchedShares = postcodeShares[
        postcodeShares.apply(
            lambda row: matchCols(gatewayRow, row, gatewayMatchColNames, gatewayMatchColNames), axis=1
        )
    ]
    pretty_print_df(matchedShares, "Matched Shares")

    matchedFlows = chainFlowVolumes[
        chainFlowVolumes.apply(
            lambda row: matchCols(gatewayRow, row, gatewayMatchColNames, gatewayMatchColNames), axis=1
        )
    ]
    pretty_print_df(matchedFlows, "Matched Flows")

    if len(matchedFlows) == 0:
        print('INPUT DATA ERROR: No chain flow volumes corresponding to this gateway\n')
        continue

    matchedTos = matchedShares\
                .merge(matchedFlows, left_on='Chain Link', right_on='To', how='right')\
                .rename(columns={'Postcode':'To Postcode'})\
                .filter(['To Postcode','From','To','From State','To State','Run Type'])
    pretty_print_df(matchedTos, "Matched Tos (After First Merge)")

    matchedFroms = matchedTos\
                .merge(matchedShares, left_on='From', right_on='Chain Link', how='left')\
                .rename(columns={'Postcode':'From Postcode'})\
                .filter(['From Postcode','From','From State','To Postcode','To','To State','Run Type'])
    pretty_print_df(matchedFroms, "Matched Froms (After Second Merge)")

    matchedTimes = matchedFroms\
                    .merge(travelTimes, on=['From Postcode', 'To Postcode'], how='left')
    pretty_print_df(matchedTimes, "Matched Times (After Travel Times Merge)")

    # Check for NaN travel times where postcodes exist
    nan_times = matchedTimes[
        (matchedTimes['Travel Time'].isna()) & 
        (matchedTimes['To Postcode'].notna())
    ]
    if len(nan_times) > 0:
        print('INPUT DATA WARNING: Routes without travel times:\n')
        pretty_print_df(nan_times, "Routes Missing Travel Times")

    # Replace NaN travel times with 4*60 (240 minutes)
    matchedTimes['Travel Time'] = matchedTimes['Travel Time'].fillna(4*60)
    pretty_print_df(matchedTimes, "Matched Times (After NaN Fill)")

    # Add gateway matching columns
    for i in range(len(gatewayMatchColNames)):
        matchedTimes[gatewayMatchColNames[i]] = gatewayRow[gatewayMatchColNames[i]]
    
    matchedTimes['Industry Class'] = gatewayRow['Industry Class']
    matchedTimes = matchedTimes[gatewayMatchColNames + 
                              ['Industry Class','Run Type','From Postcode','From',
                               'From State','To Postcode','To','To State','Travel Time']]
    pretty_print_df(matchedTimes, "Final Matched Times for this Gateway")

    # Use pd.concat instead of append
    allTravelTimes = pd.concat([allTravelTimes, matchedTimes])
    pretty_print_df(allTravelTimes, "Cumulative allTravelTimes")
    break

# Final output
pretty_print_df(allTravelTimes, "Final Complete allTravelTimes")


Current gateway:['Swanson Dock West' 'I&MC' 'Import' 'Full' 'IMEX' 4209.470819953]


Matched Shares


,Gateway,Sector,Import / Export,Full / Empty,Chain Link,Freight Area,Postcode,Y2056
0,*,*,*,*,Swanson Dock West,Port of Melbourne - Swanson Dock West,3003,1.0
1,*,*,*,*,Swanson Dock East,Port of Melbourne - Swanson Dock East,3003,1.0
2,*,*,*,*,Webb Dock North,Port of Melbourne - Webb Dock North,3207,1.0
3,*,*,*,*,Webb Dock South,Port of Melbourne - Webb Dock South,3207,1.0
4,*,*,*,*,Appleton Dock,Port of Melbourne - Appleton Dock,3003,1.0
5,*,*,*,*,Dynon North Rail Terminal,Dynon North Rail Terminal,3003,1.0
6,*,*,*,*,Dynon South Rail Terminal,Dynon South Rail Terminal,3003,1.0
7,*,*,*,*,Altona Rail Terminal,Altona Rail Terminal,3018,1.0
8,*,*,*,*,WIFT West Rail Terminal,WIFT West Rail Terminal,3029,1.0
9,*,*,*,*,WIFT East Rail Terminal,WIFT East Rail Terminal,3029,1.0


Shape: (80, 8)
Showing first 10 rows of 80 total rows

Matched Flows


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Volume Entry,From,To,From State,To State,Run Type,Output Link Share
0,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442
1,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671
2,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977
3,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909
4,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.085009
5,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Complete,IG,0.036433
6,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals Others,Importers,Full,Empty,CR,0.075304
7,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals Others,Importers,Full,Complete,IG,0.008367
8,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Transport Depots,Importers,Full,Empty,CR,0.617909
9,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Importers,PRS Terminals WIFT / BIFT,Empty,Empty,CR,0.034800


Shape: (17, 12)
Showing first 10 rows of 17 total rows

Matched Tos (After First Merge)


...

Shape: (2098, 13)
Showing first 10 rows of 2098 total rows

Cumulative allTravelTimes


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,From,From State,To Postcode,To,To State,Travel Time
0,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3029,PRS Terminals WIFT / BIFT,Full,27.543957
1,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3029,PRS Terminals WIFT / BIFT,Full,27.543957
2,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3753,PRS Terminals WIFT / BIFT,Full,44.769091
3,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3753,PRS Terminals WIFT / BIFT,Full,44.769091
4,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3018,PRS Terminals Others,Full,18.705176
5,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3975,PRS Terminals Others,Full,37.665978
6,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3062,PRS Terminals Others,Full,28.902344
7,Swanson Dock West,I&MC,Import,Full,IMEX,CR,3003,Swanson Dock West,Full,3003,Importers,Empty,1.856365
8,Swanson Dock West,I&MC,Import,Full,IMEX,CR,3003,Swanson Dock West,Full,3011,Importers,Empty,11.612198
9,Swanson Dock West,I&MC,Import,Full,IMEX,CR,3003,Swanson Dock West,Full,3012,Importers,Empty,14.160464


Shape: (2098, 13)
Showing first 10 rows of 2098 total rows

Final Complete allTravelTimes


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,From,From State,To Postcode,To,To State,Travel Time
0,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3029,PRS Terminals WIFT / BIFT,Full,27.543957
1,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3029,PRS Terminals WIFT / BIFT,Full,27.543957
2,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3753,PRS Terminals WIFT / BIFT,Full,44.769091
3,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3753,PRS Terminals WIFT / BIFT,Full,44.769091
4,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3018,PRS Terminals Others,Full,18.705176
5,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3975,PRS Terminals Others,Full,37.665978
6,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3062,PRS Terminals Others,Full,28.902344
7,Swanson Dock West,I&MC,Import,Full,IMEX,CR,3003,Swanson Dock West,Full,3003,Importers,Empty,1.856365
8,Swanson Dock West,I&MC,Import,Full,IMEX,CR,3003,Swanson Dock West,Full,3011,Importers,Empty,11.612198
9,Swanson Dock West,I&MC,Import,Full,IMEX,CR,3003,Swanson Dock West,Full,3012,Importers,Empty,14.160464


Shape: (2098, 13)
Showing first 10 rows of 2098 total rows


In [11]:
allTravelTimes

,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,From,From State,To Postcode,To,To State,Travel Time
0,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3029,PRS Terminals WIFT / BIFT,Full,27.543957
1,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3029,PRS Terminals WIFT / BIFT,Full,27.543957
2,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3753,PRS Terminals WIFT / BIFT,Full,44.769091
3,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3753,PRS Terminals WIFT / BIFT,Full,44.769091
4,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3003,Swanson Dock West,Full,3018,PRS Terminals Others,Full,18.705176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3458,Exporters,Full,3062,Transport Depots,Full,71.550223
277,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3803,Exporters,Full,3062,Transport Depots,Full,55.483854
278,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,BR,3753,Transport Depots,Full,3753,BIFT South Rail Terminal,Complete,15.981749
279,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,BR,3061,Transport Depots,Full,3753,BIFT South Rail Terminal,Complete,25.706089
